# Lab 9 - Sentiment Analysis con RNNs

El objetivo de este laboratorio es entrenar un clasificador de reseñas de películas utilizando una red neuronal recurrente (RNN). Para ello, se utilizará el dataset de IMDB, que contiene 50,000 reseñas de películas etiquetadas como positivas o negativas.

## Preparación del entorno.

Si no estamos parados en el repo, clonar y cd al repo. Esto nos permite usar el mismo notebook tanto local como en Google Colab.

In [ ]:
import os

REPO_NAME = "lab9"
if REPO_NAME not in os.getcwd():
  if not os.path.exists(REPO_NAME):
    !git clone https://github.com/FCEIA-AAII/{REPO_NAME}.git
  os.chdir(REPO_NAME)


Importar librerías

In [ ]:
import numpy as np
from pathlib import Path
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt

Establecer GPU por defecto en caso de estar disponible.

In [ ]:
# Configurar para que TensorFlow utilice la GPU por defecto
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Configurar para que TensorFlow asigne memoria dinámicamente
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # Especificar la GPU por defecto
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Manejar error
        print(e)

Cargar dataset:

In [ ]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

Inspeccionar dataset:

In [ ]:
train_dataset.element_spec

Vemos que el primer elemento es un tf.string, que contiene la reseña de la película. El segundo elemento es un tf.int64, que contiene la etiqueta de la reseña (0 para negativa, 1 para positiva).

Inspeccionamos algunos ejemplos:

In [ ]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

Shuffle y batching:

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 256

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


Mostrar un batch de ejemplos:

In [ ]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

Creamos un tokenizador:

In [ ]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)

only_text = train_dataset.map(lambda text, label: text)

# La función adapt ajusta el vocabulario al texto, debe ser llamada con un dataset de texto
encoder.adapt(only_text)

Mostramos los primeros 20 tokens:

Los dos primeros tokens son los de padding y desconocido, respectivamente. Los siguientes tokens son los más comunes en el dataset.

In [ ]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

Una vez que tenemos el tokenizador, podemos convertir las reseñas a tokens:

In [ ]:
encoded_example = encoder(example)[0].numpy()
encoded_example

Construimos el siguiente modelo:

![model.png](model.png)

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Compilamos el modelo:

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Entrenamos

In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

Definimos una función para plotear métricas:

In [ ]:
def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

Evaluamos métricas:

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

Corremos sobre reseñas de ejemplo:

In [ ]:
sample_text = ('The movie was cool. The animation and the graphics were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))

print(sample_text)
print(predictions)

sample_text = ('An awful movie. A lackluster plot, uninspired performances, and forgettable action sequences.')
predictions = model.predict(np.array([sample_text]))

print(sample_text)
print(predictions)

Probar agregando una segunda capa LSTM bidireccional, la primer capa debe devolver secuencias (argumento `return_sequences=True`).